# FHIR Digitally Signing FHIR Bundle or QuestionnaireResponse Object

This is a Jupyter Notebook using Python 3.7 and openSSl to create JSON Web Signature (JWS)(see RFC 7515) and attach it to a FHIR Bundle or QuestionnaireResponse resource.

- If the resource is a Bundle use Bundle.signature
- If the resource is a QuestionnaireResponse use its [Signature extension](http://hl7.org/fhir/StructureDefinition/questionnaireresponse-signature
)

See signatures: http://build.fhir.org/signatures.html

**For IG Publishing option to remove the signature from the Signature element and let the IG publisher sign and verify it  (currently the IG publisher is messing up the narrative so it is tricky to predict the narrative)**

*Although self-signed certificates are used for the purpose of these examples, they are not recommended for production systems.*

### Import Libraries

In [1]:
from pprint import pprint
from requests import get, post
from json import dumps, loads
from yaml import dump as dumpy, load as loady, SafeLoader
from pathlib import Path
from datetime import datetime
import pytz
from jose import jws  #python JWS package
from base64 import b64encode
from jcs import canonicalize #package for a JCS (RFC 8785) compliant canonicalizer.
from cryptography import x509
from cryptography.hazmat.backends import default_backend
from lxml import etree
from copy import deepcopy

/Users/ehaas/.pyenv/versions/fhir_builds/lib/python3.7/site-packages/jose/backends/cryptography_backend.py:4: CryptographyDeprecationWarning: Python 3.7 is no longer supported by the Python core team and support for it is deprecated in cryptography. The next release of cryptography will remove support for Python 3.7.
  from cryptography.exceptions import InvalidSignature, InvalidTag


### 1. Set up Parameters

 #### 1.01 Choose the FHIR Bundle or QuestionnaireResponse File to Sign

In [2]:
local_file = True
remove_nested_for_pub = True # if publication rule is to remove all nested meta profile  - simple case for nested entries  TODO update for all nested meta.profiles
detached = True # Option to make a detached signature. (default to True for FHIR)
remove_sig_for_pub = True ## # if publication rule is to remove all nested meta profile  - simple case for nested entries  TODO update for all nested meta.profiles
detached = True # Option to make a detached signature. (default to True for FHIR)

payload_id = 'cdex-document-digital-sig-example' # can be a Bundle or Questionnaire
# payload_id = 'cdex-searchbundle-digital-sig-example' # can be a Bundle or Questionnaire
# payload_id = 'cdex-questionnaireresponse-example4' # can be a Bundle or Questionnaire
payload_id

in_path = '/Users/ehaas/Documents/FHIR/davinci-ecdx/input/examples-yaml'

out_path = 'out_files'  #YAML only
out_path = in_path  #YAML only
out_file_name = 'signed_object' #YAML only
out_file_name = payload_id #YAML only

# my_url = "https://argopatientlist.aidbox.app/fhir/Bundle"
# my_url = 'http://test.fhir.org/r4/Bundle'
my_url = 'https://hl7.org/fhir/us/davinci-cdex'

auto_timestamp = False
timezone= 'US/Pacific'

#### 1.02. Define the location of the Document Signing Certificate

 - See the Jupyter file [Create_Cert.ipynb]() for how to generate your own self-signed certificate.

In [3]:
certificate_path = Path('example_org_cert') # update this to your folder
cert_pem = certificate_path / 'cert.pem'
cert_der = certificate_path / 'cert.der'
certificate = x509.load_der_x509_certificate(cert_der.read_bytes(), default_backend())
certificate

<Certificate(subject=<Name(C=US,ST=California,L=Sausalito,O=Example Organization,CN=John Hancock\, MD,1.2.840.113549.1.9.1=jhancock@example.org)>, ...)>

### 3. Create JWS to Attach to Bundle or QuestionnaireResponse

#### 3.01  Auto Timestamp Function

In [4]:
def get_timestamp(timezone):  # Get current time in timezone
  tz = pytz.timezone('US/Pacific')
  current_time = datetime.now(tz)
  iso_timestamp = current_time.isoformat()  # Format as ISO 8601 compliant string
  return(iso_timestamp)
# print(get_timestamp(timezone))

#### 3.04 Extract Common Name (CN) from the Subject as String

In [5]:
def get_cn():
  for attr in certificate.subject.get_attributes_for_oid(x509.NameOID.COMMON_NAME):
      cn_raw = attr.value  # e.g., "John Hancock, MD"
      # Check if CN is an email address (contains '@')
      if '@' not in cn_raw:
          cn = cn_raw
          break  # Use the first non-email CN
  return(cn)
# print(get_cn())

#### 3.06 Extract Subject Key Identifier (KID) as Hexadecimal String

In [6]:
def get_kid():
    kid_ext = certificate.extensions.get_extension_for_class(x509.SubjectKeyIdentifier)
    kid = kid_ext.value.digest.hex() # Hexadecimal string
    return(kid)
# print(get_kid())

#### 3.08 Extract Subject Alternative Name (SAN) as Dict

- format
  -  "otherName" = 2.16.840.1.113883.4.6;UTF8:9941339108
  -  "URI" = https://example.org/fhir/Practitioner/123

In [7]:
def get_san():
  san = {}
  san['NPI'] = []
  san_ext = certificate.extensions.get_extension_for_class(x509.SubjectAlternativeName)
  san['DNS'] = san_ext.value.get_values_for_type(x509.DNSName)
  san['OtherName'] = san_ext.value.get_values_for_type(x509.OtherName)
  san['URI'] = san_ext.value.get_values_for_type(x509.UniformResourceIdentifier)
  for other_name in san['OtherName']:
      if other_name.type_id.dotted_string == '2.16.840.1.113883.4.6':
        san['NPI'].append(other_name.value.decode('utf-8').replace('\x0c\n',''))
  return san
# print(get_san()['NPI'][0])

#### 3.09 Canonicalize the XHTML in the Narrative using C14 N 1.0

In [8]:
def canonicalize_xhtml(narrative_div):

  parser = etree.XMLParser(remove_blank_text=True)
  root = etree.fromstring(narrative_div, parser)
  # Canonicalize the XML using C14N 1.0
  canonicalized_str = etree.tostring(root, method="c14n", with_comments=False).decode('utf-8')
  return canonicalized_str

# narrative_div = '<div xmlns="http://www.w3.org/1999/xhtml"> Acute Asthmatic attack. Was wheezing\n\n\n\n                        for days prior to admission. </div>'
# canonicalize_xhtml(narrative_div)


#### 3.1. Prepare Header

 note the base64 DER is Cert PEM file wihout the footer and header and line returns

In [9]:
der = cert_pem.read_text()
der = der.replace('-----BEGIN CERTIFICATE-----','')
der = der.replace('-----END CERTIFICATE-----','')
der = der.replace('\n','')
header = {
    "alg": "RS256",
    "kty": "RS",
    "srCms": [
    {
      "commId":{
      "id": "urn:oid:1.2.840.10065.1.12.1.5",
      "desc": "Verification Signature"
    },
      "commQuals": ["Verification of medical record integrity"]
    }
    ],
    "sigT": get_timestamp(timezone) if auto_timestamp else '2020-10-23T04:54:56.048+00:00',
    "kid": get_kid(),
    "x5c": [der],
     }
header

{'alg': 'RS256',
 'kty': 'RS',
 'srCms': [{'commId': {'id': 'urn:oid:1.2.840.10065.1.12.1.5',
    'desc': 'Verification Signature'},
   'commQuals': ['Verification of medical record integrity']}],
 'sigT': '2020-10-23T04:54:56.048+00:00',
 'kid': 'bfbe3e5c0470444576548113928d5f1e4e3f2eeb',
 'x5c': ['MIIFVzCCA7+gAwIBAgIUBn43F4O110zVNHPUtBnnXf33FQwwDQYJKoZIhvcNAQELBQAwgZUxCzAJBgNVBAYTAlVTMRMwEQYDVQQIDApDYWxpZm9ybmlhMRIwEAYDVQQHDAlTYXVzYWxpdG8xHTAbBgNVBAoMFEV4YW1wbGUgT3JnYW5pemF0aW9uMRkwFwYDVQQDDBBKb2huIEhhbmNvY2ssIE1EMSMwIQYJKoZIhvcNAQkBFhRqaGFuY29ja0BleGFtcGxlLm9yZzAeFw0yNTA2MjUyMzEyMzlaFw0yNzA2MTUyMzEyMzlaMIGVMQswCQYDVQQGEwJVUzETMBEGA1UECAwKQ2FsaWZvcm5pYTESMBAGA1UEBwwJU2F1c2FsaXRvMR0wGwYDVQQKDBRFeGFtcGxlIE9yZ2FuaXphdGlvbjEZMBcGA1UEAwwQSm9obiBIYW5jb2NrLCBNRDEjMCEGCSqGSIb3DQEJARYUamhhbmNvY2tAZXhhbXBsZS5vcmcwggGiMA0GCSqGSIb3DQEBAQUAA4IBjwAwggGKAoIBgQCcXmvX60GA5G+Dl4iRn9TS/wTF1FTH9RmrP29G6XSOuVDEgGZwUTJI/OkRLPj+JUKy/kMY3Ym41k3JRr8NrJ7Ucjf3Te2Y0zmRMfGKO2X7p01Id8rGhnbsTkWjszcckjKOTk7E4HXO7XQ

### 3.2 Fetch Payload from FHIR Server or Local File
- **set local_file to "True" for local file**

In [10]:
def fetch_payload():

  # url = "https://argopatientlist.aidbox.app/fhir/Bundle"
  # url = 'http://test.fhir.org/r4/Bundle'
  url = 'https://hl7.org/fhir/us/davinci-cdex'

  username = "basic"
  password = "secret"
  headers = {"Accept": "application/fhir+json" , "Content-Type": "application/fhir+json"}

  r = get(f'{url}/{payload_id}.json', auth=(username, password), headers = headers)
  my_obj = r.json()
  # print("="*80)
  # print("STATUS: ",r.status_code)

  # print("="*80)
  # print("HEADERS:\n")
  # for k,v in r.headers.items():
  #     print(f'{k} = {v}')
  # print("="*80)
  # print("BODY:\n")
  # print(dumps(my_obj,indent=2))
  return(my_obj)


if local_file: 
  path = Path() / in_path / f'{payload_id}.yml'
  my_obj =loady(path.read_text(), Loader=SafeLoader)
else:
  my_obj = fetch_payload()
# print(dumpy(my_obj,indent=2, sort_keys=False))
my_obj

{'resourceType': 'Bundle',
 'identifier': {'system': 'urn:ietf:rfc:3986',
  'value': 'urn:uuid:c173535e-135e-48e3-ab64-38bacc68dba8'},
 'type': 'document',
 'timestamp': '2021-10-25T20:16:29-07:00',
 'entry': [{'fullUrl': 'urn:uuid:17a80a8d-4cf1-4deb-a1fd-2db1130e5f76',
   'resource': {'resourceType': 'Composition',
    'id': '17a80a8d-4cf1-4deb-a1fd-2db1130e5f76',
    'text': {'status': 'generated',
     'div': '<div xmlns="http://www.w3.org/1999/xhtml"><a name="Composition_17a80a8d-4cf1-4deb-a1fd-2db1130e5f76"> </a><h3>Medical Records Document</h3><p>This document, titled "Active Conditions," was created on October 25, 2021, by Dr. John Hancock, who also legally attested to its accuracy on the same date. It summarizes the active medical conditions for a patient based on an emergency encounter on October 25, 2021. The document includes one section detailing an active condition: Type 2 Diabetes Mellitus.</p></div>'},
    'status': 'final',
    'type': {'coding': [{'system': 'http://loi

#### 3.2.1 Prepare Payload

The payload is the base64_url form of the canonicalized version of the resource before attaching the signature

1. Creat a deepcopy 
2. Canonicalize the narrative(s) (xhtml) using the C14N 1.0 specification (http://www.w3.org/2006/12/xml-c14n10).
3. remove bundle.id, .meta .signature elements per Subscriptions, optionally remove nested meta profiles per publication rules
4. if publication rule is to remove all nested meta profile - option to remove all nested meta profiles,  (only simple case for nested entries - TODO update for all nested meta.profiles)
5. For FHIR Document - insert anchors for publisher of form <a name="[Recourse_Type]_[fullUrl]"> </a> into the deepcopy for verification.
6. Canonicalize the resource using IETF JSON Canonicalization Scheme (JCS)

In [11]:
if my_obj['resourceType'] in ['Bundle', 'QuestionnaireResponse']:
  deep_copy = deepcopy(my_obj)
  deep_copy.pop('id', None)
  deep_copy.pop('meta', None)
  deep_copy.pop('signature', None)
else:
  print('Not a Bundle or QuestionnaireResponse')

if remove_nested_for_pub and deep_copy['resourceType'] == 'Bundle':
  print (" ======= Removed nested meta profile from Bundle! ===========")
  for bundle_entry in deep_copy['entry']:
      try:
         entry_profile = bundle_entry['resource']['meta'].pop('profile', None)
      except KeyError as e:
        print(f"KeyError no {e} at entry - {bundle_entry['resource']['resourceType']} FullUrl: {bundle_entry['fullUrl']}")
      else:
        # print(entry_profile)
        if not bundle_entry['resource']['meta']:
          bundle_entry['resource'].pop('meta')
        # else:
        #   print(bundle_entry['resource']['meta'])
      # if deep_copy['type'] == "document": #For FHIR Document - insert anchors for publisher
      #     entry_anchor = f'<a name="{bundle_entry["resource"]["resourceType"]}_{bundle_entry["resource"]["id"]}"> </a>'
      #     xhtml_declaration = '<div xmlns="http://www.w3.org/1999/xhtml">'
      #     bundle_entry['resource']['text']['div'] = bundle_entry['resource']['text']['div'].replace(xhtml_declaration,xhtml_declaration + entry_anchor)
      #     print(bundle_entry['resource']['text']['div'])
if deep_copy['resourceType'] == 'QuestionnaireResponse':
  try:
    for i, extension in enumerate(deep_copy['extension']):
       if extension['url'] == 'http://hl7.org/fhir/StructureDefinition/questionnaireresponse-signature':
          deep_copy_signature_ext = deep_copy['extension'].pop(i) # remove element
    if i == 0:
      deep_copy.pop('extension') # remove extension if empty
  except KeyError:
    print('No signature extension found')

payload = canonicalize(deep_copy)
print(payload)
print(len(payload))

try:
  deep_copy['text']['div'] = canonicalize_xhtml(deep_copy['text']['div'])
except KeyError as e:
  print(f"KeyError no {deep_copy['resourceType']}.{e}")
try:
  for bundle_entry in deep_copy['entry']:
    try:
      print(f"Length before xhtml canonicalization for entry {bundle_entry['resource']['resourceType']}: {len(bundle_entry['resource']['text']['div'])}")
      bundle_entry['resource']['text']['div'] = canonicalize_xhtml(bundle_entry['resource']['text']['div'])
      print(f"Length after xhtml canonicalization for entry {bundle_entry['resource']['resourceType']}: {len(bundle_entry['resource']['text']['div'])}")
    except KeyError as e:
      print(f"KeyError no bundle_entry['resource'][{e}]")
      continue
except KeyError as e:
  print(f"KeyError no {deep_copy['resourceType']}.{e}")
payload = canonicalize(deep_copy)
payload, len(payload)

 ======= Removed nested meta profile from Bundle! ===========
KeyError no 'meta' at entry - Composition FullUrl: urn:uuid:17a80a8d-4cf1-4deb-a1fd-2db1130e5f76
b'{"entry":[{"fullUrl":"urn:uuid:17a80a8d-4cf1-4deb-a1fd-2db1130e5f76","resource":{"attester":[{"mode":"legal","party":{"display":"Example Practitioner","reference":"urn:uuid:0820c16d-91de-4dfa-a3a6-f140a516a9bc"},"time":"2021-10-25T20:16:29-07:00"}],"author":[{"display":"Example Practitioner","reference":"urn:uuid:0820c16d-91de-4dfa-a3a6-f140a516a9bc"}],"date":"2021-10-25T20:16:29-07:00","encounter":{"display":"Example Encounter","reference":"urn:uuid:5ce5c83a-000f-47d2-941c-039358cc9112"},"id":"17a80a8d-4cf1-4deb-a1fd-2db1130e5f76","resourceType":"Composition","section":[{"entry":[{"reference":"urn:uuid:014a68ec-d691-49e0-b980-91b0d924e570"}],"title":"Active Condition 1"}],"status":"final","subject":{"display":"Example Patient","reference":"urn:uuid:970af6c9-5bbd-4067-b6c1-d9b2c823aece"},"text":{"div":"<div xmlns=\\"http://www.

(b'{"entry":[{"fullUrl":"urn:uuid:17a80a8d-4cf1-4deb-a1fd-2db1130e5f76","resource":{"attester":[{"mode":"legal","party":{"display":"Example Practitioner","reference":"urn:uuid:0820c16d-91de-4dfa-a3a6-f140a516a9bc"},"time":"2021-10-25T20:16:29-07:00"}],"author":[{"display":"Example Practitioner","reference":"urn:uuid:0820c16d-91de-4dfa-a3a6-f140a516a9bc"}],"date":"2021-10-25T20:16:29-07:00","encounter":{"display":"Example Encounter","reference":"urn:uuid:5ce5c83a-000f-47d2-941c-039358cc9112"},"id":"17a80a8d-4cf1-4deb-a1fd-2db1130e5f76","resourceType":"Composition","section":[{"entry":[{"reference":"urn:uuid:014a68ec-d691-49e0-b980-91b0d924e570"}],"title":"Active Condition 1"}],"status":"final","subject":{"display":"Example Patient","reference":"urn:uuid:970af6c9-5bbd-4067-b6c1-d9b2c823aece"},"text":{"div":"<div xmlns=\\"http://www.w3.org/1999/xhtml\\"><a name=\\"Composition_17a80a8d-4cf1-4deb-a1fd-2db1130e5f76\\"> </a><h3>Medical Records Document</h3><p>This document, titled \\"Active C

##### Then base64_url the payload entry

note this step is combined with 3.3 below using the jws.sign method.

#### 3.3 Create Signature using private key and the RS256 algorithm to get the JWS compact serialization format

note the signature is displayed with the parts labeled and separated with line breaks for easier viewing.

In [12]:
private_key_path = certificate_path / 'private-key.pem'
private_key = private_key_path.read_text()
private_key

signature = jws.sign(payload,private_key,algorithm='RS256',headers=header)

labels = ['header', 'payload', 'signature']
for i,j in enumerate(signature.split('.')):
    print(f'{labels[i]}:\n{j}\n')

header:
eyJhbGciOiJSUzI1NiIsImtpZCI6ImJmYmUzZTVjMDQ3MDQ0NDU3NjU0ODExMzkyOGQ1ZjFlNGUzZjJlZWIiLCJrdHkiOiJSUyIsInNpZ1QiOiIyMDIwLTEwLTIzVDA0OjU0OjU2LjA0OCswMDowMCIsInNyQ21zIjpbeyJjb21tSWQiOnsiZGVzYyI6IlZlcmlmaWNhdGlvbiBTaWduYXR1cmUiLCJpZCI6InVybjpvaWQ6MS4yLjg0MC4xMDA2NS4xLjEyLjEuNSJ9LCJjb21tUXVhbHMiOlsiVmVyaWZpY2F0aW9uIG9mIG1lZGljYWwgcmVjb3JkIGludGVncml0eSJdfV0sInR5cCI6IkpXVCIsIng1YyI6WyJNSUlGVnpDQ0E3K2dBd0lCQWdJVUJuNDNGNE8xMTB6Vk5IUFV0Qm5uWGYzM0ZRd3dEUVlKS29aSWh2Y05BUUVMQlFBd2daVXhDekFKQmdOVkJBWVRBbFZUTVJNd0VRWURWUVFJREFwRFlXeHBabTl5Ym1saE1SSXdFQVlEVlFRSERBbFRZWFZ6WVd4cGRHOHhIVEFiQmdOVkJBb01GRVY0WVcxd2JHVWdUM0puWVc1cGVtRjBhVzl1TVJrd0Z3WURWUVFEREJCS2IyaHVJRWhoYm1Odlkyc3NJRTFFTVNNd0lRWUpLb1pJaHZjTkFRa0JGaFJxYUdGdVkyOWphMEJsZUdGdGNHeGxMbTl5WnpBZUZ3MHlOVEEyTWpVeU16RXlNemxhRncweU56QTJNVFV5TXpFeU16bGFNSUdWTVFzd0NRWURWUVFHRXdKVlV6RVRNQkVHQTFVRUNBd0tRMkZzYVdadmNtNXBZVEVTTUJBR0ExVUVCd3dKVTJGMWMyRnNhWFJ2TVIwd0d3WURWUVFLREJSRmVHRnRjR3hsSUU5eVoyRnVhWHBoZEdsdmJqRVpNQmNHQTFVRUF3d1FTbTlvYmlCSVlXNWpiMk5y

#### 3.4. Option to Create "detached-content" payload by removing the payload from the JWS

note the signature is displayed with the parts labeled and separated with line breaks for easier viewing then as compact serialization format

In [13]:
if detached:
  split_sig = signature.split('.')
  split_sig[1] = ''
  signature = '.'.join(split_sig)
  for i,j in enumerate(signature.split('.')):
      print(f'{labels[i]}:\n{j}\n')

print(f'\nSignature in compact serialization format:\n{"="*80}\n{signature}')

header:
eyJhbGciOiJSUzI1NiIsImtpZCI6ImJmYmUzZTVjMDQ3MDQ0NDU3NjU0ODExMzkyOGQ1ZjFlNGUzZjJlZWIiLCJrdHkiOiJSUyIsInNpZ1QiOiIyMDIwLTEwLTIzVDA0OjU0OjU2LjA0OCswMDowMCIsInNyQ21zIjpbeyJjb21tSWQiOnsiZGVzYyI6IlZlcmlmaWNhdGlvbiBTaWduYXR1cmUiLCJpZCI6InVybjpvaWQ6MS4yLjg0MC4xMDA2NS4xLjEyLjEuNSJ9LCJjb21tUXVhbHMiOlsiVmVyaWZpY2F0aW9uIG9mIG1lZGljYWwgcmVjb3JkIGludGVncml0eSJdfV0sInR5cCI6IkpXVCIsIng1YyI6WyJNSUlGVnpDQ0E3K2dBd0lCQWdJVUJuNDNGNE8xMTB6Vk5IUFV0Qm5uWGYzM0ZRd3dEUVlKS29aSWh2Y05BUUVMQlFBd2daVXhDekFKQmdOVkJBWVRBbFZUTVJNd0VRWURWUVFJREFwRFlXeHBabTl5Ym1saE1SSXdFQVlEVlFRSERBbFRZWFZ6WVd4cGRHOHhIVEFiQmdOVkJBb01GRVY0WVcxd2JHVWdUM0puWVc1cGVtRjBhVzl1TVJrd0Z3WURWUVFEREJCS2IyaHVJRWhoYm1Odlkyc3NJRTFFTVNNd0lRWUpLb1pJaHZjTkFRa0JGaFJxYUdGdVkyOWphMEJsZUdGdGNHeGxMbTl5WnpBZUZ3MHlOVEEyTWpVeU16RXlNemxhRncweU56QTJNVFV5TXpFeU16bGFNSUdWTVFzd0NRWURWUVFHRXdKVlV6RVRNQkVHQTFVRUNBd0tRMkZzYVdadmNtNXBZVEVTTUJBR0ExVUVCd3dKVTJGMWMyRnNhWFJ2TVIwd0d3WURWUVFLREJSRmVHRnRjR3hsSUU5eVoyRnVhWHBoZEdsdmJqRVpNQmNHQTFVRUF3d1FTbTlvYmlCSVlXNWpiMk5y

### 4. base64 the JWS and Add/Update the Signature element to the Bundle or QR
   
- this is what would be contained and/or referenced by TASK over-the-wire

In [14]:
b64_jws = b64encode(signature.encode()).decode()
sig_element = {
            "type": [  # The Signature.type shall contain the same values as the srCms element."
              {
                "system": "urn:iso-astm:E1762-95:2013",
                "code": header['srCms'][0]['commId']['id'].replace('urn:oid:',''), # copy from header
                "display": header['srCms'][0]['commId']['desc'], # copy from header
              }
            ],
            "when": get_timestamp(timezone) if auto_timestamp else '2020-10-23T04:54:56.048+00:00', #system timestamp when signature created
            # "who" { #Reference to the Practitioner who signed and attested to the Bundle
            #   "reference": "Practitioner/123"  
            #   "display": "Practitioner/123" 
            # },
            # "onBehalfOf": { #Reference to the Organization
            #   "reference": "Organization/123"
            #   "display": "Organization/123"
            # },
            "who": { #Reference to the Practitioner who signed and attested to the Bundle using NPI should be same as SAN
                    "identifier": {
                        "system": "http://hl7.org/fhir/sid/us-npi",
                        "type" : {
                            "coding" : [{
                                "system" : "http://terminology.hl7.org/CodeSystem/v2-0203",
                                "code" : "NPI"
                              }]
                          },
                        "value": get_san()['NPI'][0] # extract SAN from certificate
                    },
                    "display": get_cn()  #  extract CN from certificate
                },
            "onBehalfOf": {
                    "identifier": {
                        "system": "http://hl7.org/fhir/sid/us-npi",
                        "value": "1184932014"
                    }
                },
            "targetFormat" : "application/fhir+json;canonicalization=http://hl7.org/fhir/canonicalization/json#document", # The technical format of the signed resources see 
            #https:// hl7.org/fhir/json.html#canonical. The MIME types can include optional parameters in the format type/subtype;
            # parameter=value, as defined in RFC 2045 and RFC 6838. For example, text/plain;charset=utf-8 specifies a character encoding.
            "sigFormat" : "application/jose", # The technical format of the signature
            "data": b64_jws
             }

if my_obj['resourceType'] == 'Bundle':
  my_obj['signature'] = sig_element # update signature
if my_obj['resourceType'] == 'QuestionnaireResponse':  # only create new extension for now TODO add ability to append to extension and check for old signatures 
    my_obj['extension'] = [dict(
        url='http://hl7.org/fhir/StructureDefinition/questionnaireresponse-signature',
        valueSignature=sig_element
        )]
try:
  print(dumps(my_obj['signature'], indent=2, sort_keys=False))
except KeyError:
  print(dumps(my_obj['extension'], indent=2, sort_keys=False))

{
  "type": [
    {
      "system": "urn:iso-astm:E1762-95:2013",
      "code": "1.2.840.10065.1.12.1.5",
      "display": "Verification Signature"
    }
  ],
  "when": "2020-10-23T04:54:56.048+00:00",
  "who": {
    "identifier": {
      "system": "http://hl7.org/fhir/sid/us-npi",
      "type": {
        "coding": [
          {
            "system": "http://terminology.hl7.org/CodeSystem/v2-0203",
            "code": "NPI"
          }
        ]
      },
      "value": "9941339100"
    },
    "display": "John Hancock, MD"
  },
  "onBehalfOf": {
    "identifier": {
      "system": "http://hl7.org/fhir/sid/us-npi",
      "value": "1184932014"
    }
  },
  "targetFormat": "application/fhir+json;canonicalization=http://hl7.org/fhir/canonicalization/json#document",
  "sigFormat": "application/jose",
  "data": "ZXlKaGJHY2lPaUpTVXpJMU5pSXNJbXRwWkNJNkltSm1ZbVV6WlRWak1EUTNNRFEwTkRVM05qVTBPREV4TXpreU9HUTFaakZsTkdVelpqSmxaV0lpTENKcmRIa2lPaUpTVXlJc0luTnBaMVFpT2lJeU1ESXdMVEV3TFRJelZEQTBPalUwT2pVMkx

### Using FHIR RESTful create POST to FHIR Server 
***Deactivated until until get new AIDBOX***
using AIDBox for now at `https://argopatientlist.aidbox.app/fhir/`
<!-- using `http://test.fhir.org/r4/` -->


### Alternatively Write to Local File as JSON and YAML

In [16]:
path = Path(r'out_files/signed_object.json') # Fixed output file for json
print(f'=========== remove_nested_for_pub = {remove_nested_for_pub} ================')
print(f'=========== detached = {detached} ================')
print(f'=========== remove_sig_for_pub = {remove_sig_for_pub} ================')
if remove_sig_for_pub:
  try:
    my_obj['signature'].pop('data')
  except KeyError:
    my_obj['extension']['valueSignauture'].pop('data') # TODO update for only signature extension
print(f'Writing signed object to {path}')
path.write_text(dumps(my_obj,indent=2,sort_keys=False))
path = Path() /out_path / f"{out_file_name}.yml"
print(f'Writing signed object to {path}')
how_to = '''
# steps to create/update dig signature examples
#
# 1. create unsigned Bundle or QuestionnaireResponse example source file with minified Narrative element(s) to prevent the publisher from adding its own.
# 1. for Documents Bundles use the autogenerated narrative instead to get it to verify. ( use only the -i parameter to keep the meta elements )
# 1. optionally create certificate using this script file: https://github.com/HL7/davinci-ecdx/blob/master/CDEX-Signatures/Create_Cert.ipynb
# 1. create signatures using this script file: https://github.com/HL7/davinci-ecdx/blob/master/CDEX-Signatures/Create_Digsign_Bundle_or_QR.ipynb
#  -  remove elements from payload as defined by the signature canonical and publisher requirements (for example, meta profile elements)
# 1. save to YAML source
# 1. run sushi and publisher again with -ink parameters
# 1. verify signatures using this script file: https://github.com/HL7/davinci-ecdx/blob/master/CDEX-Signatures/Verify_digsign_Bundle_or_QR.ipynb
'''

path.write_text(f'{how_to}\n{dumpy(my_obj,indent=2,sort_keys=False)}')
if payload_id == 'cdex-document-digital-sig-example':  # update the parameters examples with the signed doc too
  path = Path() / in_path / 'cdex-parameters-example2.yml'
  print(f'Writing signed object to {path}')
  params =loady(path.read_text(), Loader=SafeLoader)
  meta_extension =  my_obj["meta"].pop("extension") 
  params['parameter'][7]['part'][2]['resource'] = my_obj
  my_obj["meta"]["extension"] = meta_extension # add  back in so is idempotent cell
  path.write_text(dumpy(params,indent=2,sort_keys=False))

=========== remove_nested_for_pub = True ================
=========== detached = True ================
=========== remove_sig_for_pub = True ================
Writing signed object to out_files/signed_object.json
Writing signed object to /Users/ehaas/Documents/FHIR/davinci-ecdx/input/examples-yaml/cdex-document-digital-sig-example.yml
Writing signed object to /Users/ehaas/Documents/FHIR/davinci-ecdx/input/examples-yaml/cdex-parameters-example2.yml
